In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import folktables
from folktables import ACSDataSource, ACSEmployment

from sklearn.preprocessing import PolynomialFeatures

In [2]:
def has_converged(x, size, epsilon):
    for i in range(0,size):
            if np.linalg.norm(x[-1:][0] - x[-(2+i):][0], np.inf) > epsilon:
                return False
    return True

In [3]:
def linear_proxies(x, y, z, iters, epsilon, eta_val = None):
    n = x.shape[0]
    theta = [LinearRegression(fit_intercept=False).fit(x,z).coef_.reshape(-1,1)]
    theta_average = [LinearRegression(fit_intercept=False).fit(x,z).coef_.reshape(-1,1)]
    z_sum = np.sum(z)
    grad_term_overall = np.sum(x, axis=0)/z_sum
    converged = False
    
    if eta_val:
        eta = eta_val
        
    print("Iterations:")    
    for t in range(1, iters):
        if eta_val is None:
            eta = 1/(np.sqrt(n*t))
            
        if t%200 == 0:
            print(t)
        
        zhat = np.matmul(x,theta[t-1])
        costs = (zhat-z)*(1-2*y)
        
        prc = LinearRegression(fit_intercept=False).fit(x,costs)
        h_real = prc.predict(x)
        h_plus = h_real > 0
        h_minus = h_real < 0
    
        if np.sum(h_real[h_plus]) > -np.sum(h_real[h_minus]):
            h = h_plus
        else:
            h = h_minus
            
        h = h.astype(int)
        zhat_sum = np.sum(zhat)
        err_points = np.abs(h-y)
        
        err_cost = np.squeeze(np.matmul((zhat-z).T,err_points))/n
        overall_diff = zhat_sum/z_sum - 1

        if np.abs(overall_diff) >= np.abs(err_cost):           
            penalty = np.sign(overall_diff) * grad_term_overall
        else:
            penalty = np.sign(err_cost) * np.matmul(np.transpose(x),err_points)/n
            
        new_theta = theta[t-1] - eta*penalty.reshape(-1,1)
        theta.append(new_theta)
        theta_average.append((t*theta_average[t-1]+new_theta)/(t+1))
        
        if t%100 == 0 :
            if has_converged(theta_average, 10, epsilon):
                converged = True
                break
                    
    return theta, theta_average[-1]   

In [4]:
def evaluate_proxy(x, y, z, theta_average):
    n = x.shape[0]
    zhat = np.matmul(x, theta_average) 
    final_costs = (zhat-z)*(1-2*y)
    prc = LinearRegression(fit_intercept=False).fit(x, final_costs)  
    h_real = prc.predict(x)
    h_plus = h_real > 0
    h_minus = h_real < 0
    
    if np.sum(h_real[h_plus]) > -np.sum(h_real[h_minus]):
        h = h_plus
    else:
        h = h_minus
    
    h = h.astype(int)    
       
    zhat_sum = np.sum(zhat)
    z_sum = np.sum(z)
    err_points = np.abs(h-y)

    err_cost_z = np.squeeze(np.matmul(z.T, err_points))
    err_cost_zhat = np.squeeze(np.matmul(zhat.T, err_points))
    overall_diff = zhat_sum/z_sum - 1
    model_diff = (err_cost_zhat - err_cost_z)/n
    proxy_error = err_cost_zhat/zhat_sum - err_cost_z/z_sum
    
    return overall_diff, model_diff, proxy_error

I. Census Data Experiments

In [5]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=False)
features, label, group = ACSEmployment.df_to_numpy(acs_data)

print(features)
print(group)
print(label)

[[30. 14.  1. ...  2.  1.  8.]
 [18. 14.  5. ...  2.  2.  1.]
 [69. 17.  1. ...  2.  1.  9.]
 ...
 [43. 16.  1. ...  2.  2.  8.]
 [23. 16.  5. ...  2.  1.  8.]
 [18. 14.  5. ...  2.  1.  8.]]
[8 1 9 ... 8 8 8]
[False False False ... False False False]


In [ ]:
bw_indices = (group<3)

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features[bw_indices,:], label[bw_indices], group[bw_indices]-1, test_size=0.30, random_state=0)

#Normalize feature matrix
#intercept = np.ones(X_train.shape[0]).reshape(-1,1)
y = y_train.astype(int).reshape(-1,1)
z = group_train.reshape(-1,1)
#x = X_train[:,:-1]/(np.linalg.norm(X_train[:,:-1], axis=1).max())
x = X_train[:,:-1]

for deg in range(1,10):
    print(deg)
    poly_reg=PolynomialFeatures(degree=deg)
    x_poly = poly_reg.fit_transform(x)
    print(x_poly)
    print(x_poly.shape)
    coefficients, theta_average = linear_proxies(x_poly, y, z, 1000, 0.0001)
    overall_diff, model_diff, proxy_error = evaluate_proxy(x_poly, y, z, theta_average)
    
    print("Polynomial Degree: " + str(deg))
    print("Overall diff: " + str(overall_diff))
    print("Model diff: " + str(model_diff))
    print("Proxy error: " + str(proxy_error))

1
[[ 1.  8.  6. ...  2.  2.  2.]
 [ 1. 48. 17. ...  2.  2.  1.]
 [ 1.  3.  1. ...  2.  0.  2.]
 ...
 [ 1. 59. 18. ...  2.  2.  2.]
 [ 1. 43. 20. ...  2.  2.  1.]
 [ 1. 62. 22. ...  2.  2.  1.]]
(175592, 16)
Iterations:
200


In [ ]:
filename = "ACS.npz"
np.savez(filename, X = x, Y = y.flatten(), grouplabels = z.T, group_sets = ["White", "Black"], group_types = "Race", is_binary = True, proxy = np.matmul(x, theta_average))

II. Synthetic Data Experimental Framework

In [ ]:
def synthetic_proxy_experiments(T, trials, n, d, epsilon, eta, unit=False):
    discrepancy_proxy = []
    discrepancy_total = []
    discrepancy_h = []
    intercept = np.ones(n).reshape(-1,1)
    
    for i in range(0,trials):
        np.random.seed(i)
        y = np.round(np.random.rand(n,1))
        
        if unit:
            theta = np.random.rand(d,1)
            theta = theta/np.linalg.norm(theta)
            x = np.random.rand(n,d)
            
            for j in range(0,n):
                x[j,:] = x[j,:]/np.linalg.norm(x[j,:])
                
            zhat = np.matmul(x,theta)
            z = np.random.binomial(n=n,p=zhat)   
            
        else:
            z = np.round(np.random.rand(n,n))
            theta = np.random.rand(n,n)
            x = np.matmul(z, np.linalg.inv(theta))
            x = np.hstack((intercept, x))
            z = z[:,0].reshape(-1,1)

        coefficients, theta_average = linear_proxies(x, y, z, T, epsilon, eta)
        overall_diff, model_diff, proxy_error = evaluate_proxy(x,y,z,theta_average)
       
        discrepancy_total.append(overall_diff)
        discrepancy_h.append(model_diff)
        discrepancy_proxy.append(proxy_error)
        
    return discrepancy_total, discrepancy_h, discrepancy_proxy

III. FTPL Algo (Non-Linear Proxies)